In [1]:
import glob
import os

import numpy as np
import pandas as pd

# setting notebook
pd.set_option("display.max_columns", None)

In [2]:
# go to data-joined folder
%cd data-joined

C:\Users\user\Desktop\gte\notebooks\data-joined


In [3]:
%run xuat_combined.ipynb 
%run po_combined.ipynb 
%run dmsp.ipynb
%run dmkh.ipynb
#go back the tree
%cd ..

C:\Users\user\Desktop\gte\notebooks


In [4]:
# make copy
xuat = xuat_joined.copy()
po = po_joined.copy()
sp = dmsp_full.copy()
kh = dmkh_full.copy()

## PO

In [5]:
#drop cols
cols_used = ['Mã phiếu đề xuất\n(Code)',
 'Mã khách hàng\n(Customer Code)',
 'Mã chủ hàng',
 'Ngày về\n(Receiving Date)',
 'Hạn chót giao hàng\n(Delivery Deadline)',
 'Ngày thực tế giao hàng',
 'Chứng từ KH ký nhận',
 'Chứng từ khác',
 'Tình trạng đơn hàng',
 'Kho xuất',
 'Loại phiếu',
 'Người giao hàng (Shipper delivery by)',
'Unnamed: 23']

po_selected_cols = po[cols_used]

In [6]:
po_replace = po_selected_cols.copy()

#Replace 'Chua thuc hien' => 'Da nhat hang'
po_replace.loc[(po_replace['Tình trạng đơn hàng'].str.contains('Chưa thực hiện')) 
                & (po_replace['Chứng từ khác'].str.contains('Đã nhặt hàng')),'Tình trạng đơn hàng'] = 'Đã nhặt hàng'

#Replace 'De xuat' => 'Don'
po_replace['Loại phiếu'] = po_replace['Loại phiếu'].str.replace(r'Đề xuất', 'Đơn', regex=True)

In [7]:
# make a copy
po_cal = po_replace.copy()
# create month
po_cal['month'] = pd.DatetimeIndex(po_cal['Ngày về\n(Receiving Date)']).month
# impute date diff invoice

diff_ct_yes = (po_cal['Chứng từ KH ký nhận'] - po_cal['Ngày thực tế giao hàng']).dt.days #da hoan
diff_ct_no = (pd.Timestamp.now().normalize() - po_cal['Ngày thực tế giao hàng']).dt.days #chua hoan

conditions = [
    po_cal['Tình trạng đơn hàng']=='Đã bàn giao chứng từ',
    po_cal['Tình trạng đơn hàng']=='Đã giao hàng'
]
choices = [diff_ct_yes, diff_ct_no]

po_cal['datediff_chungtu'] = np.select(conditions, choices, default=np.nan)

# impute date diff delivery

diff_deli = (po_cal['Ngày thực tế giao hàng'] - po_cal['Ngày về\n(Receiving Date)']).dt.days
po_cal['datediff_giaohang'] = np.where(diff_deli<0, np.nan, diff_deli)

In [8]:
po_filter = po_cal.copy()
#Filter Warehouse not contains
values_to_remove = ['Hyperion','hư hỏng','xử lý','Hư Hỏng','khuyến mãi','Tồn hàng','hàng hóa', 'Kho treo hàng']

pattern = '|'.join(values_to_remove)

po_filter_kho = po_filter.loc[~po_filter['Kho xuất'].str.contains(pattern, case=False)]
#Filter Ma chu hang HP, RW
values_to_remove_mch = ['HP', 'RW', 'GB'] 

pattern_mch = '|'.join(values_to_remove_mch)

po_filter_mch = po_filter_kho.loc[po_filter_kho['Mã chủ hàng'].str.contains(pattern_mch, case=False)]
po_processed = po_filter_mch.copy()

## BCKT

In [9]:
# merge
xuat_dmsp = xuat.merge(sp, left_on='Mã vật tư\n(Goods Code)', right_on='PRODUCT CODE', how='left')
xuat_dmsp['CBM/Unit'] = xuat_dmsp['CBM/Unit']*xuat_dmsp['Số lượng\n(Quantily)']
xuat_dmsp['GrossWeightProductg'] = xuat_dmsp['GrossWeightProductg']*xuat_dmsp['Số lượng\n(Quantily)']
# aggregate 
xuat_dmsp = xuat_dmsp.groupby(['Mã phiếu đề xuất', 'Mã chủ hàng_x']).agg(
      {
        'CBM/Unit': 'sum',
        'GrossWeightProductg': 'sum',
        'Số lượng\n(Quantily)': 'sum',
        'Mã vật tư\n(Goods Code)': 'count',
        'Tiền hàng\n(Sub Total)': 'sum'
      }
)
# rename cols
dict_rename = {
    'CBM/Unit': 'cbm_po',
     'GrossWeightProductg': 'kg_po',
     'Số lượng\n(Quantily)': 'unit_po',
     'Mã vật tư\n(Goods Code)': 'sku_po',
     'Tiền hàng\n(Sub Total)': 'amount_po'
}

xuat_dmsp.rename(columns=dict_rename, inplace=True)
xuat_processed = xuat_dmsp.copy()

## Create data

In [10]:
# Create data region
data = [
    ["Kho Hà Nội", "Miền Bắc"],
    ["Kho Phạm Văn Đồng", "Miền Nam"],
    ["Kho Bình Tân", "Miền Nam"],
    ["Kho Bùi Thị Xuân", "Miền Nam"],
    ["Kho Đà Nẵng", "Miền Trung"],
    ["Kho Thuận An", "Miền Nam"],
    ["Kho Bình Hòa", "Miền Nam"],
    ["Kho thuê dịch vụ - Daso", "Miền Nam"],
    ["Kho Woojin Bắc Ninh", "Miền Bắc"],
    ["Kho LogisValley - Long Biên", "Miền Bắc"],
    ["Kho thuê ngoài Emskip Q9", "Miền Nam"],
    ["Kho LogisticValey - Long Biên", "Miền Bắc"],
    ["Kho Hateco Long Biên", "Miền Bắc"],
    ["Kho Gamuda Hà Nội", "Miền Bắc"],
    ["Kho thuê dịch vụ Daso", "Miền Nam"],
    ['Kho LogisticValey Long Biên',"Miền Bắc"],
    ['Kho Online PVD', 'online'],
    ['Kho Upbase Nơ Trang Long', 'online'],
    ['Kho Online Nơ Trang Long', 'online'],
    ['Kho Online Hà Nội', 'online'],
    ['Kho Online Upbase Hà Nội', 'online'],
    ['Kho Online Upbase PVD', 'online'],
    ['Kho Online Đà Nẵng','online']
]

# Create the pandas DataFrame
df_rgn = pd.DataFrame(data, columns=["Kho", "Region"])
data_in_sub = [
    ["Kho Hà Nội", "Hà Nội", "Nội thành"],
    ["Kho Phạm Văn Đồng", "Hồ Chí Minh", "Nội thành"],
    ["Kho Bình Tân", "Hồ Chí Minh", "Nội thành"],
    ["Kho Bùi Thị Xuân", "Bình Dương", "Nội thành"],
    ["Kho Đà Nẵng", "Đà Nẵng", "Nội thành"],
    ["Kho Thuận An", "Bình Dương", "Nội thành"],
    ["Kho Bình Hòa", "Hồ Chí Minh", "Nội thành"],
    ["Kho thuê dịch vụ - Daso", "Hồ Chí Minh", "Nội thành"],
    ["Kho Woojin Bắc Ninh", "Hà Nội", "Nội thành"],
    ["Kho LogisValley - Long Biên", "Hà Nội", "Nội thành"],
    ["Kho thuê ngoài Emskip Q9", "Hồ Chí Minh", "Nội thành"],
    ["Kho LogisticValey - Long Biên", "Hà Nội", "Nội thành"],
    ["Kho Hateco Long Biên", "Hà Nội", "Nội thành"],
    ["Kho Gamuda Hà Nội", "Hà Nội", "Nội thành"],
    ["Kho thuê dịch vụ Daso", "Hồ Chí Minh", "Nội thành"],
    ['Kho LogisticValey Long Biên',"Hà Nội", "Nội thành"]
]

# Create the pandas DataFrame
df_in_sub = pd.DataFrame(data_in_sub, columns=["Kho xuất", "Des", "in_sub"])

## Merge

In [11]:
# merge po - bckt
po_xuat = po_processed.merge(
    xuat_processed, left_on=['Mã phiếu đề xuất\n(Code)', 'Mã chủ hàng'], right_on=['Mã phiếu đề xuất', 'Mã chủ hàng_x'], how="left")
# merge region
po_region = po_xuat.merge(df_rgn, left_on="Kho xuất", right_on="Kho", how="left").drop('Kho', axis=1)
# merge DMKH
po_kh = po_region.merge(
    kh[['Customer Code', 'Account', 'Des']], left_on="Mã khách hàng\n(Customer Code)", right_on="Customer Code", how="left").drop('Customer Code', axis=1)
# merge inner/suburbs data
po_insub = po_kh.merge(df_in_sub, on=["Kho xuất", "Des"], how="left")
po_insub.loc[~po_insub.Des.isna(), "in_sub"] = po_insub.in_sub.fillna("Ngoại thành")
conditions = [
    po_insub["datediff_chungtu"] < 0,
    po_insub["datediff_chungtu"] <= 6,
    po_insub["datediff_chungtu"] <= 14,
    po_insub["datediff_chungtu"] <= 30,
    po_insub["datediff_chungtu"] > 30,
]
choices = ["Invalid_days", "0 - 6 ngày", "7 - 14 ngày", "15 - 30 ngày", "Trễ"]

po_insub["day_level"] = np.select(conditions, choices, default=np.nan)
po_run = po_insub.copy()
po_run['Mã phiếu đề xuất\n(Code)'] = po_run['Mã phiếu đề xuất\n(Code)'] + po_run['Mã chủ hàng']
dict_rename = {
    "Mã phiếu đề xuất\n(Code)": "ID_PO",
    "Mã khách hàng\n(Customer Code)": "ID_cus",
    "Ngày về\n(Receiving Date)": "date_receive",
    "Hạn chót giao hàng\n(Delivery Deadline)": "deli_dl",
    "Tiền hàng\n(Sub Total)": "amount_po",
    'Unnamed: 23':'ngaybangiaohoadon'
}

po_run.rename(columns=dict_rename, inplace=True)

In [12]:
# import datetime
# file_path = '../data/processed/'
# namefile = f'PO_{datetime.datetime.now().strftime("%H%M_%Y%m%d")}.csv'
# po_run.to_csv(file_path+namefile, index=False, encoding="utf-8-sig")


In [13]:
# po_run[po_run.Region.isnull()]['Kho xuất'].unique()

In [14]:
po_run['ngaybangiaohoadon'] = np.where(~po_run['ngaybangiaohoadon'].astype(str).str.contains(r'^\d{2}\/\d{2}\/\d{4}', regex=True, na=False), pd.NaT, po_run['ngaybangiaohoadon'])
po_run['ngaybangiaohoadon'] = po_run['ngaybangiaohoadon'].apply(pd.to_datetime, format='%d/%m/%Y')

## chung tu (chua loc hp)

In [15]:
chungtu = pd.pivot_table(po_run, index='Tình trạng đơn hàng', values='ID_PO', aggfunc='count')

In [16]:
chungtu['%'] = chungtu['ID_PO'] / chungtu['ID_PO'].sum() *100

In [17]:
chungtu

,ID_PO,%
Tình trạng đơn hàng,,
Chưa giao hàng,2039,0.928134
Chưa thực hiện,2406,1.095190
PO hủy,8734,3.975638
Đã bàn giao chứng từ,199636,90.872510
Đã giao hàng,6710,3.054332
Đã nhặt hàng,163,0.074196


In [18]:
chungtu_chuahoan = po_run[(po_run['Tình trạng đơn hàng'] == 'Đã giao hàng')|(po_run['Tình trạng đơn hàng'] == 'Đã bàn giao chứng từ')]

chungtu_chuahoan = pd.pivot_table(chungtu_chuahoan, index='day_level', columns='Tình trạng đơn hàng', values='ID_PO', aggfunc='count')

chungtu_chuahoan['%_dahoan'] = chungtu_chuahoan['Đã bàn giao chứng từ'] / chungtu_chuahoan['Đã bàn giao chứng từ'].sum() *100
chungtu_chuahoan['%_chuahoan'] = chungtu_chuahoan['Đã giao hàng'] / chungtu_chuahoan['Đã giao hàng'].sum() *100

chungtu_chuahoan

Tình trạng đơn hàng,Đã bàn giao chứng từ,Đã giao hàng,%_dahoan,%_chuahoan
day_level,,,,
0 - 6 ngày,141739.0,3026.0,70.998718,45.096870
15 - 30 ngày,10164.0,577.0,5.091266,8.599106
7 - 14 ngày,35515.0,2853.0,17.789878,42.518629
Invalid_days,1496.0,1.0,0.749364,0.014903
Trễ,10166.0,253.0,5.092268,3.770492
nan,556.0,NaN,0.278507,NaN


---

In [19]:
gtri_ctchuahoan = po_run[(po_run['Tình trạng đơn hàng'] == 'Đã giao hàng')]

In [20]:

pd.pivot_table(gtri_ctchuahoan, index='day_level', columns='Tình trạng đơn hàng', values='amount_po', aggfunc=np.sum).reset_index()

Tình trạng đơn hàng,day_level,Đã giao hàng
0,0 - 6 ngày,1.151174e+10
1,15 - 30 ngày,1.777042e+09
2,7 - 14 ngày,9.031630e+09
3,Invalid_days,0.000000e+00
4,Trễ,9.145429e+08


In [21]:
pd.pivot_table(gtri_ctchuahoan, index='Region', columns='Tình trạng đơn hàng', values='amount_po', aggfunc=np.sum).reset_index()

Tình trạng đơn hàng,Region,Đã giao hàng
0,Miền Bắc,1.056397e+10
1,Miền Nam,1.134048e+10
2,Miền Trung,1.153940e+09
3,online,1.765706e+08


---

In [22]:
ctdahoan = po_run[(po_run['Tình trạng đơn hàng'] == 'Đã bàn giao chứng từ')]

In [23]:
pd.pivot_table(ctdahoan, index='Region', columns='day_level', values='ID_PO', aggfunc='count').reset_index()

day_level,Region,0 - 6 ngày,15 - 30 ngày,7 - 14 ngày,Invalid_days,Trễ,nan
0,Miền Bắc,40544.0,4676.0,13945.0,122.0,797.0,33.0
1,Miền Nam,59252.0,3491.0,16399.0,162.0,709.0,143.0
2,Miền Trung,7244.0,759.0,2365.0,29.0,206.0,NaN
3,online,34699.0,1238.0,2806.0,1183.0,8454.0,380.0


---

## van hanh

In [24]:
po_summa = po_run.copy()

In [25]:
po_summa['Người giao hàng (Shipper delivery by)'] = po_summa['Người giao hàng (Shipper delivery by)'].str.lower()

In [26]:
cond = po_summa['Người giao hàng (Shipper delivery by)'].str.contains('nhất|viet|ker|xe tải ngoài|taixe thau hcm', na=False)
po_summa['dvngoai'] = po_summa.loc[cond, 'Người giao hàng (Shipper delivery by)']

In [76]:
po_summa.to_csv('../data/processed/po-meeting.csv',encoding='utf-8-sig', index=False)

## load data

In [28]:
meeting = pd.read_csv('../data/processed/po-meeting.csv', encoding='utf-8-sig')

In [29]:
meeting_xuatban = meeting[(meeting['Loại phiếu'] == 'Đơn bán')].reset_index(drop=True)

In [69]:
meeting_xuatban['Tình trạng đơn hàng'].unique()

array(['Đã bàn giao chứng từ', 'PO hủy', 'Đã giao hàng', 'Chưa thực hiện',
       'Chưa giao hàng', 'Đã nhặt hàng'], dtype=object)

In [31]:
# Define the aggregation calculations
aggregations = {
    'date_receive': ['count', 'nunique'],
    'Ngày thực tế giao hàng': ['count', 'nunique'],
    'datediff_giaohang': 'median',
    'cbm_po': 'median',
    'kg_po': 'median',
    'unit_po': 'median',
    'sku_po': 'median',
    'dvngoai': 'count'
}
summa1 = meeting_xuatban.groupby(['Mã chủ hàng', 'Region'], as_index=False, dropna=False).agg(aggregations)

In [32]:
summa1.columns = ['_'.join(cols).lower()
                     for cols in summa1.columns.to_flat_index()]

In [33]:
summa1['po_ve1'] = summa1['date_receive_count']/summa1['date_receive_nunique']
summa1['po_giao1'] = summa1['ngày thực tế giao hàng_count']/summa1['ngày thực tế giao hàng_nunique']
summa1['pct_dvngoai'] = summa1['dvngoai_count']/summa1['date_receive_count']

In [34]:
summa1

,mã chủ hàng_,region_,date_receive_count,date_receive_nunique,ngày thực tế giao hàng_count,ngày thực tế giao hàng_nunique,datediff_giaohang_median,cbm_po_median,kg_po_median,unit_po_median,sku_po_median,dvngoai_count,po_ve1,po_giao1,pct_dvngoai
0,GB,Miền Bắc,4,4,4,4,1.0,0.0859,22605.75,118.5,11.0,0,1.000000,1.000000,0.000000
1,GB,Miền Nam,17,9,17,7,0.0,0.0568,17424.00,52.0,13.0,0,1.888889,2.428571,0.000000
2,GB,online,39966,339,39788,330,2.0,0.0024,384.00,4.0,1.0,2,117.893805,120.569697,0.000050
3,HP,Miền Bắc,57168,288,53460,345,3.0,0.0290,3975.00,60.0,4.0,22710,198.500000,154.956522,0.397250
4,HP,Miền Nam,69058,291,65196,322,3.0,0.0246,3444.00,52.0,3.0,16479,237.312715,202.472050,0.238626
5,HP,Miền Trung,10532,270,9509,265,3.0,0.0216,2582.50,42.0,3.0,3290,39.007407,35.883019,0.312381
6,HP,online,1571,125,1511,117,0.0,0.0096,2580.00,47.0,7.0,0,12.568000,12.914530,0.000000
7,HP01,online,902,98,867,82,1.0,0.0030,2120.00,2.0,1.0,0,9.204082,10.573171,0.000000
8,RW,Miền Bắc,2533,271,2497,278,2.0,0.0216,3491.00,24.0,2.0,449,9.346863,8.982014,0.177260
9,RW,Miền Nam,5370,288,5317,291,1.0,0.0239,4740.00,24.0,3.0,384,18.645833,18.271478,0.071508


In [35]:
meeting_xuatban_filter_gb = meeting_xuatban[(meeting_xuatban['Tình trạng đơn hàng'] == 'Đã bàn giao chứng từ')|(meeting_xuatban['Tình trạng đơn hàng'] == 'Đã giao hàng')].reset_index(drop=True)

In [36]:
meeting_gb = meeting_xuatban_filter_gb[meeting_xuatban_filter_gb['Mã chủ hàng'] == 'GB'].reset_index()

In [37]:
meeting_gb['amount_po'].median()

89090.5

In [38]:
meeting_gb['amount_po'].mean()

158725.83666460076

In [39]:
# create a list of our conditions
conditions = [
    (meeting_gb['amount_po'] < 100000),
    (meeting_gb['amount_po'] < 200000),
    (meeting_gb['amount_po'] < 300000),
    (meeting_gb['amount_po'] >= 300000)
    ]

# create a list of the values we want to assign for each condition
values = ['duoi100', '100-200', '200-300', 'tren300']

# create a new column and use np.select to assign values to it using our lists as arguments
meeting_gb['amountlevel'] = np.select(conditions, values)

In [40]:
# Define the aggregation calculations
aggregations = {
    'ID_PO': 'count'
}
meeting_gb.groupby(['amountlevel'], as_index=False, dropna=False).agg(aggregations)

,amountlevel,ID_PO
0,100-200,10760
1,200-300,3057
2,duoi100,21084
3,tren300,3823


In [83]:
# Define the aggregation calculations
aggregations = {
    'date_receive': ['count', 'nunique'],
    'cbm_po': 'median',
    'kg_po': 'median',
    'unit_po': 'median',
    'sku_po': 'median',
    'amount_po': 'sum'
}
summa_gb = meeting_gb.groupby(['Mã chủ hàng'], as_index=False, dropna=False).agg(aggregations)

In [84]:
summa_gb.columns = ['_'.join(cols).lower()
                     for cols in summa_gb.columns.to_flat_index()]

In [108]:
round(meeting_gb[meeting_gb['Region']=='online']['amount_po'].describe())

count       38703.0
mean       149813.0
std        270127.0
min             0.0
25%         52778.0
50%         89090.0
75%        158582.0
max      21526361.0
Name: amount_po, dtype: float64

In [110]:
round(meeting_gb[meeting_gb['Region']=='online']['amount_po'].std())

270127